# Action Recognition @ UCF101  

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import random 
import time

import torch
import torchvision
import torchvision.transforms as transforms

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import minmax_scale
import random
import torchvision.models as models


In [5]:
import torch
# torch.cuda.is_available()
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu"  
device = torch.device(dev)  
print(device)

cuda:0


---
---
## **Problem 1.** Feature extraction

In [3]:
import pandas as pd
actions = pd.read_csv('annos/actions.txt',sep='  ',header = None)
actions.columns = ['category_num','category_name']
labels_subsets = pd.read_csv('annos/videos_labels_subsets.txt',sep='\t',header=None)
labels_subsets.columns = ['video_name','category_num','train/test']
# labels_subsets.merge(labels_subsets,actions, on='category_num')
labels_subsets_sample = labels_subsets[labels_subsets['category_num']<26]
train_df = labels_subsets_sample[labels_subsets_sample['train/test']==1].reset_index(drop=True)
test_df = labels_subsets_sample[labels_subsets_sample['train/test']==2].reset_index(drop=True)

C:\Users\Vignesh Ram Nithin\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [6]:

vgg16 = models.vgg16(pretrained=True)
features = list(vgg16.classifier.children())[:-6] # Remove last layer
vgg16.classifier = nn.Sequential(*features)
vgg16.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
import torch
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import os

class CustomDataset(Dataset):
    def __init__(self, data, isTrain,images_path,transform=None):
        self.data = data
        self.images_path = images_path
        self.labels = self.data['category_num']
#         self.height = 48
#         self.width = 48
        self.transform = transform
        self.transform_label = transforms.Compose([transforms.ToTensor()])
        self.isTrain = isTrain

    def __getitem__(self, index):
        print(index)
        # This method should return only 1 sample and label 
        # (according to "index"), not the whole dataset
        # So probably something like this for you:
        
        video_name = self.data['video_name'][index]
        video_label = self.data['category_num'][index]
        images_name = []
        
        for i in os.listdir(os.path.join(self.images_path,video_name)):
            images_name.append(os.path.join(self.images_path,video_name,i))

        imgs = [Image.open(i) for i in images_name]

        images=[]

        transformed_imgs = []
        if self.transform is not None:
            for k in range(25):
#                 print(imgs[k].size)
                transformed_imgs.append(self.transform(imgs[k]))
        
        label = [torch.Tensor(video_label) for i in range(125)]
        
        return transformed_imgs, label

    def __len__(self):
        return len(self.labels)

images_path = 'images'

train_transform = transforms.Compose([

    transforms.FiveCrop((224,224)),
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
    transforms.Lambda(lambda crops: torch.stack([transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]) (crop) for crop in crops])),

])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])


train_dataset = CustomDataset(train_df,1,images_path,train_transform)
test_dataset = CustomDataset(test_df,0,images_path,train_transform)
batch_size = 1

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)
       

In [ ]:
features = []

for i,(data,label) in enumerate(train_loader):
    if i<1800:
        continue
#     elif i>=1800:
#         break

    data = torch.stack(data).squeeze(1).to(device)
    with torch.no_grad():
        for i in range(25):
            features.append(vgg16.eval()(data[i]))

In [ ]:
import pickle
file1 = open('train_features_1800-2408.pkl', 'wb')
# # file3 = open('train_features_1800-2408.pkl','wb')
# # file2 = open('train_features_900-1799.pkl','wb')
pickle.dump( features, file1)                      
file1.close()

In [ ]:
import pickle

file1 = open('train_features_0-899.pkl', 'rb')
train1 = pickle.load(file1)
file1.close()
file2 = open('train_features_900-1799.pkl','rb')
train2 = pickle.load(file2)
file2.close()
file = open('train_features_1800-.pkl','rb')
train = pickle.load(file)
file.close()

file = open('test_features.pkl','rb')
test = pickle.load(file)
file.close()


In [ ]:
train = train1+train2+train3

In [ ]:
train = torch.stack(train)
test = torch.stack(test)

In [ ]:
#prepare inputs for the LSTM network
#taking average of the five crops and obtaining a one single feature vector for each of the image
train = torch.mean(train,1)
test = torch.mean(test,1)

In [ ]:
#reshaping the features vectors for grouping the 25 images with time stamps for an action

train = train.view(int(train.shape[0]/25),25,train.shape[1])
test = test.view(int(test.shape[0]/25),25,test.shape[1])


In [ ]:
import pickle
file1 = open('train_features_final.pkl','wb')
pickle.dump( train, file1)
file1.close()
file2 = open('test_features_final.pkl','wb')
pickle.dump(test,file2)
file2.close()

***
***
## **Problem 2.** Modelling

In [1]:
import pickle
file = open('train_features_final.pkl','rb')
train = pickle.load(file)
file.close()

file = open('test_features_final.pkl','rb')
test = pickle.load(file)
file.close()

In [2]:
print('Shape of training data is :', train.shape)
print('Shape of test/validation data is :', test.shape)

Shape of training data is : torch.Size([2409, 25, 4096])
Shape of test/validation data is : torch.Size([951, 25, 4096])


In [43]:
import torch
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import os

class CustomLSTMdata(Dataset):
    def __init__(self, data,labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):

        input_data = self.data[index]
        label =self.labels[index]
        
        return input_data,label

    def __len__(self):
        return len(self.labels)

train_dataset = CustomLSTMdata(train,train_df['category_num'])
test_dataset = CustomLSTMdata(test,test_df['category_num'])
batch_size = 32

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1)


In [88]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(4096,256,batch_first=True)
        self.fc1 = nn.Linear(256,25)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x,states = self.lstm(x)
        x = (x[:,-1,:])
        x = self.dropout(x)
        x = self.fc1(x)
        return x

In [89]:
model = LSTM()
print(model)

LSTM(
  (lstm): LSTM(4096, 256, batch_first=True)
  (fc1): Linear(in_features=256, out_features=25, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [90]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable


def trainLSTM(model,train,labels,num_epochs=5):
    model.to(device)
    criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
#     optimizer_ft = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)
    optimizer_ft = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = Variable(inputs.to(device,dtype=torch.float)), Variable(labels.to(device,dtype=torch.long))

            # zero the parameter gradients
            optimizer_ft.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            labels = labels-1
            
            loss = criterion(outputs, labels)
#             print(loss)
            loss.backward()
            optimizer_ft.step()

            # print statistics
            running_loss += loss.item()
        print(epoch,running_loss)

    print('Finished Training')
    return model

model_trained = trainLSTM(model,train_loader,train_df['category_num'])

0 104.86027604341507
1 43.782252103090286
2 28.192734837532043
3 18.437121704220772
4 12.184323690831661
Finished Training


In [97]:
from sklearn.svm import LinearSVC

train_svm = train.view(train.shape[0],train.shape[1]*train.shape[2])
test_svm = test.view(test.shape[0],test.shape[1]*test.shape[2])
print(train_svm.shape)
clf = LinearSVC(random_state=5,penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=0.08, multi_class='ovr', fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, max_iter=10000)
clf.fit(train_svm.cpu(),train_df['category_num'])


torch.Size([2409, 102400])


LinearSVC(C=0.008, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=10000,
          multi_class='ovr', penalty='l2', random_state=5, tol=0.0001,
          verbose=0)

---
---
## **Problem 3.** Evaluation

In [91]:
from sklearn.metrics import accuracy_score
def evaluate(model,test):
    actual_label = []
    pred_label = []

    for i in test:
        img,label = i
        with torch.no_grad():
            images,labels = Variable(img.to(device,dtype=torch.float)), Variable(label.to(device,dtype=torch.long))
            model = model.float()
            pred = model(images)
            _, predicted = torch.max(pred, 1)
            pred = [predicted[j].item()+1 for j in range(len(labels))]
            actual_label.extend(label)
            pred_label.extend(pred)
    return (accuracy_score(actual_label,pred_label))

test_accuracy_LSTM = evaluate(model_trained,test_loader)
train_accuracy_LSTM = evaluate(model_trained,train_loader)

In [98]:
train_pred = clf.predict(train_svm.cpu())
test_pred = clf.predict(test_svm.cpu())

 ### The train and test accuracy of LSTM model

In [49]:
print('Training accuracy is %2.3f:' %(100.00),train_accuracy_LSTM)
print('Test accuracy is %2.3f :' %(100.00),test_accuracy_LSTM )
# lr = 0.001
# batch_size = 32

Training accuracy is 100.000: 0.9867164798671648
Test accuracy is 100.000 : 0.8128286014721346


 ### The train and test accuracy of SVM 

In [99]:
train_svm_accuracy = accuracy_score(train_pred,train_df['category_num'])
test_svm_accuracy = accuracy_score(test_pred,test_df['category_num'])


print('Training accuracy is %2.3f :' %(100.00), train_svm_accuracy )
print('Test accuracy is %2.3f :' %(100.00), test_svm_accuracy )

Training accuracy is 100.000 : 1.0
Test accuracy is 100.000 : 0.825446898002103
